In [1]:
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../src/")
%pwd

'/home/jovyan/nfs-home/scalable_ml_pipelines/src'

In [3]:
import socket

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
name_space = "eabraham-373705"

# Master node
kubernetes_master_url = "k8s://https://10.32.7.103:6443"

# Resource settings
driver_cores = "8"
executor_cores = "8"
driver_memory = "30g"
executor_memory = "30g"
executor_memory_overhead = "2g"

# These are the limits
cpu_limit = "3"  # 12 cores
memory_limit = "32g"  # Upto 32 GB
executor_limit = "8"

In [5]:
from pyspark.sql import SparkSession

APP_NAME = 'scalables_executor'


spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(kubernetes_master_url)\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", executor_cores)\
    .config("spark.executor.memory", executor_memory)\
    .config("spark.memory.fraction", "0.7")\
    .config("spark.memory.storageFraction", "0.3")\
    .config("spark.kubernetes.executor.limit.cores", executor_limit)\
    .config("spark.kubernetes.namespace", name_space)\
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.deleteOnTermination", "false") \
    .config("spark.kubernetes.container.image.pullPolicy", "Always") \
    .config("spark.kubernetes.container.image", "node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705-v4-executor")\
    .config("spark.local.dir", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .getOrCreate()


23/12/14 17:26:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/14 17:26:26 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
spark

In [8]:
from us_used_cars_ml_pipeline.config.configuration import ConfigurationManager
from us_used_cars_ml_pipeline.components.data_conversion import DataConversion

config_manager = ConfigurationManager()
data_conversion_config = config_manager.get_data_conversion_config()

data_conversion = DataConversion(data_conversion_config)

data_conversion_config

[2023-12-14 17:26:33,241: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/config/config.yaml loaded successfully]
[2023-12-14 17:26:33,243: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/params.yaml loaded successfully]
[2023-12-14 17:26:33,245: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/schema.yaml loaded successfully]


DataConversionConfig(path_to_csv_data='hdfs:///home/eabraham-373705/data.csv', path_to_parquet_data='hdfs:///home/eabraham-373705/data/raw/raw_data.parquet')

In [9]:
%%time
data_conversion.run_stage(spark, is_new_data=False)

23/12/14 17:26:38 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CPU times: user 59.8 ms, sys: 21 ms, total: 80.7 ms
Wall time: 5min 36s
